# Split/Chunk text
markdown splitter
smaller chunks
semantic chunking

In [1]:
# !pip install -qU langchain-text-splitters

In [2]:
from langchain_text_splitters import HTMLHeaderTextSplitter
import pandas as pd
from bs4 import BeautifulSoup
import os
import re

#### Load the files

In [3]:
def load_files(path):
    file_list = os.listdir(path)
    new_list = [] # a list to store each airline
    pattern = r'scraped_(.+)\.txt'
    filenames = []
    for file in file_list:
        filename = re.search(pattern, file).group(1)
        filenames.append(filename)
        with open(f"{path}/scraped_{filename}.txt") as f:
            file = f.read().split('|,|,|\n|') # split to each page
            file.remove('')
            string_list = [] # a list to store each airline XPATHS
            for string in file:
                str_xpath = string.split('\t|')
                string_list.extend(str_xpath)
            new_list.append(string_list)

    file_dict = {index: value for index, value in enumerate(filenames)}

    return new_list, file_dict


In [4]:
file_list, file_dict = load_files('/Users/kay/Desktop/nlp/nlp_airline_project/policy_data/scraped_txt')

#### Clean, Split each file by headers, and transform to table

In [5]:
def parse_html(list):
    '''
    Input: a list of strings, each string is an XPATH with class and div
    Output: a list of strings, each string is an XPATH without class and div
    '''
    clean_text = []
    for string in list:
        # Parse the HTML string
        soup = BeautifulSoup(string, 'html.parser')

        # find table and transform it to string
        table = soup.find('table')
        if table != None: 
        
            table_text = ""
            for row in soup.find_all('tr'):
                for cell in row.find_all(['th', 'td']):
                    table_text += cell.get_text() + " "
                table_text += "\n"
                
            table_text = '<p>' + table_text + '</p>'
            

            # Find the position of the table in the soup
            position = 1

            for element in soup.descendants:
                if element == table:
                    break
                position += 1


            # Drop the table element from the soup
            table.extract()

            # turn soup to a list of descendent
            descendants_list = [str(descendant) for descendant in soup.descendants]
            # add new table to the original position
            descendants_list.insert(position, table_text)
            # join the list back a string
            joined_string = ''.join(descendants_list)
            # parse it as a soup for further processing
            soup_new = BeautifulSoup(joined_string, 'html.parser')
            soup_new = soup

        else: 
            soup_new = soup

        # cleaned string
        cleaned_content = ""
        for content in soup_new.find_all(True):
            cleaned_content += str(content)
        clean_text.append(cleaned_content)
                
    return clean_text
   

In [6]:
# testing: it works for all but components[1]
# parse_html(file_list[0])

In [7]:
def split_header(list):
    headers_to_split_on = [
        ("h1", "Header 1"),
        ("h2", "Header 2"),
        ("h3", "Header 3"),
        ("h4", "Header 4"),
        ("h5", "Header 5")
    ]

    splitted_list = []
    for string in list:
        html_splitter = HTMLHeaderTextSplitter(headers_to_split_on=headers_to_split_on)
        html_header_splits = html_splitter.split_text(string)
        splitted_list.extend(html_header_splits)

    return splitted_list

In [8]:
# split_header(parse_html(file_list[0]))

In [9]:
def to_table(list):
    df = pd.DataFrame()
    for string in list:
        dict = string.metadata
        dict['content'] = string.page_content
        df_dict = pd.DataFrame([dict])
        df = pd.concat([df, df_dict], ignore_index=True)
    
    return df

In [10]:
# to_table(split_header(parse_html(file_list[0])))

## Apply to all files and store the tables in a list for further processing

In [11]:
df_list = []
for file in file_list:
    # get index
    index = file_list.index(file)
    # processing
    airline = parse_html(file) # parse all urls (XPATHs) for an airline
    airline_split = split_header(airline) # to documents
    df_airline = to_table(airline_split)

    # adjust column order
    for column in df_airline.columns:
        if column == 'content':
            content_column = df_airline.pop('content')  # Remove 'Content' column and store it
            df_airline['content'] = content_column 

    # append airline name
    df_airline['Airline'] = file_dict[index]

    # append dataframe
    df_list.append(df_airline)

len(df_list)

10

## File based adjustment
1. Air France

In [12]:
# df_list[0]

In [13]:
df_list[0] = df_list[0].drop(df_list[0][~pd.isna(df_list[0]['Header 4'])].index) 
df_list[0] = df_list[0].drop(df_list[0][pd.isna(df_list[0]['Header 1'])].index) # drop rows that have values in header 4 since it's actually footer

In [14]:
# df_list[0]['Header 1'].unique()

2. Air Korean

In [15]:
# df_list[1]

In [16]:
df_list[1] = df_list[1].drop(df_list[1][(df_list[1]['Header 1'] == None)].index) 
df_list[1] = df_list[1].drop(df_list[1][df_list[1]['Header 2'].isin(['Go to SNS', 'Go to App'])].index) # clean checked bag

In [17]:
# df_list[1]

3. Air Singapore

In [18]:
# df_list[2]

4. Air Qatar

In [19]:
# df_list[3]

5. Air Turkish

In [20]:
# df_list[4]

In [21]:
# clean empty lines
df_list[4] = df_list[4].drop(df_list[4][(pd.isna(df_list[4]['Header 1']) & pd.isna(df_list[4]['Header 2']) | (df_list[4]['Header 3'] == 'Checked baggage calculation tool'))].index) 
# refill header 1
df_list[4].loc[(pd.isna(df_list[4]['Header 1'])) | (df_list[4]['Header 1'] == ''), 'Header 1'] = df_list[4].loc[(pd.isna(df_list[4]['Header 1'])) | (df_list[4]['Header 1'] == ''), 'Header 2']

In [22]:
# df_list[4]['Header 1'].unique()

6. Japan Airline

In [23]:
# df_list[5]

In [24]:
df_list[5] = df_list[5].drop(df_list[5][pd.isna(df_list[5]['Header 1'])].index)

In [25]:
# df_list[5]['Header 1'].unique()

7. Emirates

In [26]:
# df_list[6]

In [27]:
df_list[6] = df_list[6].drop(df_list[6][pd.isna(df_list[6]['Header 1']) | pd.isna(df_list[6]['Header 1']) & pd.isna(df_list[6]['Header 2']) & pd.isna(df_list[6]['Header 3'])].index)

In [28]:
# df_list[6]['Header 1'].unique()

8. EVA

In [29]:
# df_list[7]

In [30]:
df_list[7] = df_list[7].drop(df_list[7][pd.isna(df_list[7]['Header 1']) & pd.isna(df_list[7]['Header 2']) & pd.isna(df_list[7]['Header 3'])].index)
df_list[7] = df_list[7].drop(df_list[7][pd.isna(df_list[7]['Header 2']) & pd.isna(df_list[7]['Header 3'])].index)

In [31]:
# df_list[7]['Header 1'].unique()

9. ANA 

In [32]:
# df_list[8]

In [33]:
df_list[8] = df_list[8].drop(df_list[8][pd.isna(df_list[8]['Header 1']) | (df_list[8]['Header 1'] == 'Search ANA')].index)

In [34]:
# df_list[8]['Header 1'].unique()

10. Cathay Pacific

In [35]:
# df_list[9]

In [36]:
df_list[9] = df_list[9].drop(df_list[9][pd.isna(df_list[9]['Header 1']) | pd.isna(df_list[9]['Header 1']) & pd.isna(df_list[9]['Header 2']) & pd.isna(df_list[9]['Header 3']) & pd.isna(df_list[9]['Header 4'])].index)
df_list[9] = df_list[9].drop(df_list[9][(df_list[9]['Header 2'] == 'Helpful links') | (df_list[9]['content'] == 'Helpful links')].index)

In [37]:
# df_list[9]['Header 1'].unique()

## Concat all dataframes

In [38]:
df_final = pd.concat(df_list, ignore_index=True) # concat 10 datasets
df_final = df_final.fillna('') # clean data to concat as string

df_final.head()

,Header 1,Header 3,Header 4,content,Airline,Header 2,Header 5
0,"Weight, size, and baggage allowance",,,Hand baggage \nChecked baggage \nHow many ba...,france,,
1,"Weight, size, and baggage allowance",What is my baggage allowance?,,Your hand baggage allowance and weight restric...,france,,
2,"Weight, size, and baggage allowance",Hand baggage dimensions,,The maximum dimensions of your hand baggage ar...,france,,
3,"Weight, size, and baggage allowance",Your hand baggage weight,,"In the Economy or Premium Economy cabins, your...",france,,
4,"Weight, size, and baggage allowance",Authorized personal items,,Soft plastic bags and packaging made with rudi...,france,,


In [39]:
# concat by columns for each row and drop duplicate columns
df_final['Concat'] = df_final['Header 3'] + '\n' + df_final['Header 4'] + '\n' + df_final['Header 5'] + '\n' + df_final['content']
df_final.drop(columns=['Header 3', 'Header 4', 'Header 5', 'content'], inplace=True)

df_final.head()


,Header 1,Airline,Header 2,Concat
0,"Weight, size, and baggage allowance",france,,\n\n\nHand baggage \nChecked baggage \nHow m...
1,"Weight, size, and baggage allowance",france,,What is my baggage allowance?\n\n\nYour hand b...
2,"Weight, size, and baggage allowance",france,,Hand baggage dimensions\n\n\nThe maximum dimen...
3,"Weight, size, and baggage allowance",france,,Your hand baggage weight\n\n\nIn the Economy o...
4,"Weight, size, and baggage allowance",france,,Authorized personal items\n\n\nSoft plastic ba...


In [40]:
# concat by rows for each Header 1 & Header 2
df_final = df_final.groupby(['Airline', 'Header 1', 'Header 2'], as_index=False)['Concat'].apply(lambda x:'\n\n'.join(x))
df_final = df_final.drop_duplicates()  
df_final.head()

,Airline,Header 1,Header 2,Concat
0,ana,Carry-On Items,,\n\n\nHome Travel Information Baggage Informat...
1,ana,Carry-On Items,Information for Carry-On Items and Onboard Rules,"\n\n\nHere, you'll find guidelines for carry-o..."
2,ana,Changes and Refunds (International Flights),,\n\n\nVoluntary Changes and Refunds Reservatio...
3,ana,Changes and Refunds (International Flights),Involuntary Changes and Refunds due to ANA's r...,\n\n\nIf flight delays or cancellations occur ...
4,ana,Changes and Refunds (International Flights),Involuntary Changes and Refunds due to Irregul...,\n\n\nCustomers who experience delays and canc...


In [41]:
# temporarily fill header 1 with header 2 if header 1 is empty
df_final.loc[(df_final['Header 1'] == ''), 'Header 1'] = df_final.loc[(df_final['Header 1'] == ''), 'Header 2']

In [42]:
df_final

,Airline,Header 1,Header 2,Concat
0,ana,Carry-On Items,,\n\n\nHome Travel Information Baggage Informat...
1,ana,Carry-On Items,Information for Carry-On Items and Onboard Rules,"\n\n\nHere, you'll find guidelines for carry-o..."
2,ana,Changes and Refunds (International Flights),,\n\n\nVoluntary Changes and Refunds Reservatio...
3,ana,Changes and Refunds (International Flights),Involuntary Changes and Refunds due to ANA's r...,\n\n\nIf flight delays or cancellations occur ...
4,ana,Changes and Refunds (International Flights),Involuntary Changes and Refunds due to Irregul...,\n\n\nCustomers who experience delays and canc...
...,...,...,...,...
332,turkish,Cabin baggage terms and materials that are all...,Cabin baggage terms and materials that are all...,\n\n\nThe baggage you are carrying must be of ...
333,turkish,Checked baggage,Checked baggage,"\n\n\nTo avoid any problems on your journey, y..."
334,turkish,"LOST, DAMAGED, AND DELAYED BAGGAGE",,\n\n\nLost Baggage Means of application for yo...
335,turkish,Materials prohibited in the cabin and in cabin...,Materials prohibited in the cabin and in cabin...,\n\n\nThere are some restrictions on items tha...


## Add top 5

#### Load files

In [43]:
new_file_list, new_file_dict = load_files('/Users/kay/Desktop/nlp/nlp_airline_project/policy_data/new_scraped_txt')

In [44]:
new_file_dict

{0: 'delta', 1: 'aa', 2: 'ua'}

In [45]:
new_file_list[1]

['\n\t\t\t<h1 itemprop="name">Checked bag policy</h1>\n\t\t\t\n\n\t\t\t<!-- Special Notices -->\n\t\t\t\n\t\t\t\t\t<section class="message-info">\n\t\t\t\t\t\t<h2 class="header">Checked bag allowances</h2>\n\t\t\t\t\t\t\n\t\t\t\t\t\t\t<p>Changes to bag allowances and fees have been updated as of February 20, 2024.</p>\n\t\t\t\t\t\t\n\t\t\t\t\t\t\t<p>Travel within / between the U.S., Puerto Rico, and U.S. Virgin Islands – 1st checked bag fee is $40 ($35 if you pay online) and the 2nd checked bag fee is $45.</p>\n\t\t\t\t\t\t\n\t\t\t\t\t\t\t<p>Travel to / from Canada, Caribbean, Mexico, Central America, and Guyana – 1st checked bag fee is $35 and the 2nd checked bag fee is $45.</p>\n\t\t\t\t\t\t\n\t\t\t\t\t\t\t<p>All bag fees are non-refundable and apply per person, at each check-in location, each way, even if you purchase or get an upgrade that includes free checked bags. If you believe you\'ve been incorrectly charged for bag fees, contact an American representative for help or file a 

#### Apply to all new files

In [46]:
new_list = []
for file in new_file_list:
    # get index
    index = new_file_list.index(file)
    # processing
    airline = parse_html(file) # parse all urls (XPATHs) for an airline
    airline_split = split_header(airline) # to documents
    df_airline = to_table(airline_split)

    # adjust column order
    for column in df_airline.columns:
        if column == 'content':
            content_column = df_airline.pop('content')  # Remove 'Content' column and store it
            df_airline['content'] = content_column 

    # append airline name
    df_airline['Airline'] = new_file_dict[index]

    # append dataframe
    new_list.append(df_airline)

len(new_list)

3

11. Delta Airline

In [47]:
# new_list[0]

In [48]:
# refill header 1 of restricted items
new_list[0].loc[new_list[0]['Header 2'] == 'These Items Are Not Allowed at Any Time', 'Header 1'] = new_list[0].loc[new_list[0]['Header 2'] == 'These Items Are Not Allowed at Any Time', 'Header 2']
new_list[0] = new_list[0].drop(new_list[0][pd.isna(new_list[0]['Header 1'])].index)

In [49]:
# new_list[0]['Header 1'].unique()

12. American Airline

In [50]:
# new_list[1]

In [51]:
# new_list[1]['content'].unique()

13. United Airlines

In [52]:
# new_list[2]

In [53]:
new_list[2] = new_list[2].drop(new_list[2][pd.isna(new_list[2]['Header 1'])].index)

In [54]:
# new_list[2]

## Concat the initial and new dataframes together

In [55]:
df_final = pd.concat([pd.concat(df_list), pd.concat(new_list)], ignore_index=True)

df_final = df_final.fillna('') # clean data to concat as string

df_final.head()

,Header 1,Header 3,Header 4,content,Airline,Header 2,Header 5
0,"Weight, size, and baggage allowance",,,Hand baggage \nChecked baggage \nHow many ba...,france,,
1,"Weight, size, and baggage allowance",What is my baggage allowance?,,Your hand baggage allowance and weight restric...,france,,
2,"Weight, size, and baggage allowance",Hand baggage dimensions,,The maximum dimensions of your hand baggage ar...,france,,
3,"Weight, size, and baggage allowance",Your hand baggage weight,,"In the Economy or Premium Economy cabins, your...",france,,
4,"Weight, size, and baggage allowance",Authorized personal items,,Soft plastic bags and packaging made with rudi...,france,,


In [56]:
# concat by columns for each row and drop duplicate columns
df_final['Concat'] = df_final['Header 3'] + '\n' + df_final['Header 4'] + '\n' + df_final['Header 5'] + '\n' + df_final['content']
df_final.drop(columns=['Header 3', 'Header 4', 'Header 5', 'content'], inplace=True)

df_final.head()

,Header 1,Airline,Header 2,Concat
0,"Weight, size, and baggage allowance",france,,\n\n\nHand baggage \nChecked baggage \nHow m...
1,"Weight, size, and baggage allowance",france,,What is my baggage allowance?\n\n\nYour hand b...
2,"Weight, size, and baggage allowance",france,,Hand baggage dimensions\n\n\nThe maximum dimen...
3,"Weight, size, and baggage allowance",france,,Your hand baggage weight\n\n\nIn the Economy o...
4,"Weight, size, and baggage allowance",france,,Authorized personal items\n\n\nSoft plastic ba...


In [57]:
# concat by rows for each Header 1 & Header 2
df_final = df_final.groupby(['Airline', 'Header 1', 'Header 2'], as_index=False)['Concat'].apply(lambda x:'\n\n'.join(x))
df_final = df_final.drop_duplicates()  
df_final.head()

,Airline,Header 1,Header 2,Concat
0,aa,Carry-on bags,1 personal item and 1 carry-on,Personal item\n\n\nYour personal item like a p...
1,aa,Carry-on bags,Know what you can carry on,\n\n\nThere are some items that can only trave...
2,aa,Carry-on bags,Special notice,\n\n\nCustomers flying Basic Economy are now a...
3,aa,Carry-on bags,You may also like...,Liquids\n\n\nOversize and overweight bags Trav...
4,aa,Checked bag policy,Checked bag allowances,\n\n\nChanges to bag allowances and fees have ...


In [58]:
# temporarily fill header 1 with header 2 if header 1 is empty
df_final.loc[(df_final['Header 1'] == ''), 'Header 1'] = df_final.loc[(df_final['Header 1'] == ''), 'Header 2']


In [59]:
df_final

,Airline,Header 1,Header 2,Concat
0,aa,Carry-on bags,1 personal item and 1 carry-on,Personal item\n\n\nYour personal item like a p...
1,aa,Carry-on bags,Know what you can carry on,\n\n\nThere are some items that can only trave...
2,aa,Carry-on bags,Special notice,\n\n\nCustomers flying Basic Economy are now a...
3,aa,Carry-on bags,You may also like...,Liquids\n\n\nOversize and overweight bags Trav...
4,aa,Checked bag policy,Checked bag allowances,\n\n\nChanges to bag allowances and fees have ...
...,...,...,...,...
435,ua,Restricted items,,\n\n\nBags \nChecked bag policy \nCarry-on b...
436,ua,Restricted items,Enable JavaScript,\n\n\nBags \nChecked bag policy \nCarry-on b...
437,ua,Restricted items,Flying on a partner airline?,\n\n\nFind helpful information if your trip in...
438,ua,Restricted items,What can you fly with?,"\n\n\nTo prevent inflight danger, many common ..."


## Simple EDA

In [63]:
grouped_counts = df_final.groupby(['Airline']).size().reset_index(name='Count')
grouped_counts

,Airline,Count
0,aa,29
1,ana,16
2,cathay,25
3,delta,24
4,emirates,31
5,eva,52
6,france,2
7,japan,93
8,korean,41
9,qatar,14


## Write to csv

In [65]:
df_final.to_csv('/Users/kay/Desktop/nlp/nlp_airline_project/policy_data/policy_chunks.csv', index=False)
print("Scraped word written to policy_chunks.csv")

Scraped word written to policy_chunks.csv
